<a href="https://colab.research.google.com/github/fppfurtado/ranqueador-cadernos-tec/blob/main/Ranqueador_de_T%C3%B3picos_TEC_Concursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
# @title Entrada de Dados
nome_arquivo = "planilha - tecnologia da informacao.xlsx"  # @param {type: "string"}
total_questoes = "20"  # @param {type: "string"}
percentual_alvo = "80"  # @param {type: "string", placeholder: "entre com o número percental (ex.: 70, 80, etc)"}

print(nome_arquivo)
print(total_questoes)
print(percentual_alvo)

planilha - tecnologia da informacao.xlsx
20
80


# Ranqueador

## Imports

In [3]:
!pip install anytree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
from anytree import Node, RenderTree
from copy import deepcopy
from google.colab import drive

## Funções

In [5]:
def carregar_dados(planilha):
    """Carrega os dados da planilha e filtra linhas válidas"""
    df = pd.read_excel(planilha)
    df = df[df['Hierarquia'].notna() | df['Índice'].notna()]  # Remove linhas vazias
    return df

In [6]:
def corrige_hierarquia_ausente(df, inicio_sequencia):
    df_corrigido = df.copy()

    filtro_linhas_sem_hierarquia = df['Hierarquia'].isna() & df['Índice'].notna()
    df_linhas_sem_hierarquia = df[filtro_linhas_sem_hierarquia]
    df_linhas_nova_hierarquia = df_linhas_sem_hierarquia

    sequencia_hierarquias_faltantes = list(range(inicio_sequencia, inicio_sequencia + len(df_linhas_sem_hierarquia)))
    df_linhas_nova_hierarquia["Hierarquia"] = [str(hierarquia).zfill(2) for hierarquia in sequencia_hierarquias_faltantes]
    indices_linhas_hierarquia_inserida = df_linhas_nova_hierarquia.index.tolist()

    df_corrigido["Hierarquia"] = df_corrigido["Hierarquia"].apply(lambda x: x if not pd.isna(x) else "")

    for i in range(len(indices_linhas_hierarquia_inserida)):
        for i_row, row in df_corrigido.iterrows():
            if indices_linhas_hierarquia_inserida[i] != indices_linhas_hierarquia_inserida[-1]:
                if indices_linhas_hierarquia_inserida[i] <= i_row < indices_linhas_hierarquia_inserida[i+1]:
                    if row["Hierarquia"]:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"] + "." + str(row["Hierarquia"])
                    else:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"]
                else:
                    break
            else:
                if i_row >= indices_linhas_hierarquia_inserida[i]:
                    if row["Hierarquia"]:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"] + "." + str(row["Hierarquia"])
                    else:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"]

    return df_corrigido

In [7]:
def inserir_caminho(raiz, caminho):
    """
    Insere uma sequência de nós em uma árvore a partir do nó raiz.
    Cria apenas os nós que ainda não existem no caminho.

    :param raiz: Nó raiz da árvore (objeto Node)
    :param caminho: Lista com os nomes dos nós no caminho
    :return: O último nó do caminho
    """
    no_atual = raiz

    for nome in caminho:
        # Verifica se o nó já existe entre os filhos
        filhos = {filho.name: filho for filho in no_atual.children}
        if nome in filhos:
            no_atual = filhos[nome]
        else:
            novo_no = Node(nome, parent=no_atual)
            no_atual = novo_no  # Atualiza para o novo nó

    return no_atual

In [8]:
def construir_arvore(df, valor_raiz="00", topico_raiz=None, perc_raiz=100):
    """Constrói a árvore hierárquica a partir do DataFrame"""
    raiz = Node(valor_raiz, topico=topico_raiz, porcentagem=perc_raiz)

    df_arvore = df.copy()
    df_arvore["Nível"] = df["Hierarquia"].apply(lambda x: len(str(x).split("."))+1  if not pd.isna(x) else 1)
    df_arvore["Caminho"] = df["Hierarquia"].apply(lambda x: str(x).split("."))

    for _, row in df_arvore.iterrows():
      node = inserir_caminho(raiz, row["Caminho"])
      node.topico = row["Índice"]
      node.porcentagem = float(row["Porcentagem"].rstrip("%"))

    return raiz

In [9]:
def ranquear_topicos(raiz):
  raiz_copia = Node(raiz.name, topico=raiz.topico, porcentagem=raiz.porcentagem)

  if raiz.children:
    ordenados = sorted(raiz.children, key=lambda x: x.porcentagem, reverse=True)
    for filho in ordenados:
      filho_ordenado = ranquear_topicos(filho)
      filho_ordenado.parent = raiz_copia

  return raiz_copia

In [73]:
def simular_prova(raiz, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido=None):
  raiz_copia = Node(raiz.topico, porcentagem=raiz.porcentagem)

  if raiz.children:
    contador_hierarquia = 1
    acumulado = 0
    alvo_relativo = (float(percentual_alvo) / 100) * raiz.porcentagem

    for filho in raiz.children:
      if acumulado >= alvo_relativo:
        break

      if topico_removido and topico_removido.lower() in filho.topico.lower():
        continue

      filho_ordenado = simular_prova(filho, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido=topico_removido)
      filho_ordenado.parent = raiz_copia
      filho_ordenado.hierarquia = contador_hierarquia
      contador_hierarquia += 1
      acumulado += filho_ordenado.porcentagem

  return raiz_copia

In [57]:
def recuperar_hierarquia(node):
  if node.parent:
    hieraraquia_pai = recuperar_hierarquia(node.parent)
    return (hieraraquia_pai + '.' if hieraraquia_pai else '') + str(node.hierarquia)
  else:
    hierarquia = str(node.hierarquia) if hasattr(node, 'hierarquia') else ''
    return hierarquia

## Main

In [116]:
df = carregar_dados(f"/content/drive/MyDrive/PROJETOS/CONCURSOS/{nome_arquivo}")
df_corrigido = corrige_hierarquia_ausente(df, inicio_sequencia=1)

arvore = construir_arvore(df_corrigido, topico_raiz="TECNOLOGIA DA INFORMAÇÃO")
arvore_ranqueada = ranquear_topicos(arvore)
arvore_simulada = simular_prova(arvore_ranqueada, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido="Mesclad")

# Mostrando a árvore
for pre, fill, node in RenderTree(arvore_simulada):
  porcentagem_relativa = round(node.porcentagem / node.parent.porcentagem * 100, 2) if node.parent else ''
  texto_porcentagem = str(node.porcentagem) + '%' + (' / ' + str(porcentagem_relativa) + '%' if porcentagem_relativa else '')
  print(f"{recuperar_hierarquia(node) + '. ' if recuperar_hierarquia(node) else ''}{node.name} ({texto_porcentagem})")

print(f"Total de Tópicos: {len(arvore_simulada.descendants) + 1}")

TECNOLOGIA DA INFORMAÇÃO (100%)
1. TI - Banco de Dados (47.46% / 47.46%)
2. Bancos de Dados Dimensionais (19.31% / 19.31%)
2.1. Conceitos de Modelagem Dimensional e Business Intelligence (5.39% / 27.91%)
2.2. Data Warehouse e Data Mart (5.09% / 26.36%)
2.3. Big Data e Data Lake (4.34% / 22.48%)
2.4. Mineração de Dados (Data Mining) (3.59% / 18.59%)
3. Sistema de Gerenciamento de Banco de Dados (SGBD) (14.82% / 14.82%)
3.1. Versões de SGBDs (7.63% / 51.48%)
3.1.1. Oracle (4.49% / 58.85%)
3.1.2. SQL Server (1.5% / 19.66%)
3.1.3. MySQL (0.75% / 9.83%)
3.2. Definições e Propriedades do SGBD (3.74% / 25.24%)
3.3. Transações (Locks, ACID, etc.) (2.54% / 17.14%)
Total de Tópicos: 14


<ipython-input-6-b4f3962d7648>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linhas_nova_hierarquia["Hierarquia"] = [str(hierarquia).zfill(2) for hierarquia in sequencia_hierarquias_faltantes]


In [114]:
drive.mount('/content/drive')
DIRETORIO_NOTEBOOK = "/content/drive/MyDrive/PROJETOS/CONCURSOS/"

# Salvar árvore em arquivo .txt
with open(f"{DIRETORIO_NOTEBOOK}/arvore - {arvore_simulada.name.lower()}.txt", 'w') as f:
    for pre, _, node in RenderTree(arvore_simulada):
      porcentagem_relativa = round(node.porcentagem / node.parent.porcentagem * 100, 2) if node.parent else ''
      texto_porcentagem = str(node.porcentagem) + '%' + (' / ' + str(porcentagem_relativa) + '%' if porcentagem_relativa else '')
      f.write(f"{recuperar_hierarquia(node) + '. ' if recuperar_hierarquia(node) else ''}{node.name} ({texto_porcentagem})\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
